In [42]:
import numpy as np
import ppxf
import ppxf_util as util
from astropy.table import Table, Column
from astropy.io import fits, ascii
from glob import glob

from SPaCT import plusminus

import matplotlib.pyplot as plt
from scipy import ndimage

In [93]:
fluxcal = fits.open('UGC04344_fluxcal.fits')
h_gal = fluxcal[0].header
lamRange1 = np.array([h_gal['CRVAL1'], 
                      h_gal['CRVAL1'] + h_gal['CDELT1'] * (h_gal['NAXIS1'] - 1)])
galaxy, logLam1, velscale = util.log_rebin(lamRange1, fluxcal[0].data[47])
FWHM_gal = 4.877
FWHM_tem = 1.5

template_files = glob('miles_models/Mun*.fits')
template_files = [i for i in template_files if float(i[-12:-5]) < 13.5]

hdu = fits.open(template_files[0])
ssp = hdu[0].data
h2 = hdu[0].header
lamRange2 = h2['CRVAL1'] + np.array([0., h2['CDELT1']*(h2['NAXIS1']-1)])

hdu = fits.open(template_files[0])
ssp = hdu[0].data
h2 = hdu[0].header
lamRange2 = h2['CRVAL1'] + np.array([0., h2['CDELT1']*(h2['NAXIS1']-1)])
sspNew, logLam2, velscale = util.log_rebin(lamRange2, ssp, velscale=velscale)
templates = np.empty((sspNew.size, len(template_files)))

ssp_rows = []

for template in template_files:
    template = template.rstrip('.fits').split('/')[1]

    spectral_range = template[0]
    IMF_type = template[1:3]
    IMF_slope = float(template[3:7])
    Z = plusminus(template[8])*float(template[9:13])
    T = float(template[14:])
    ssp_i = [template, spectral_range, IMF_type, IMF_slope, Z, T]
    ssp_rows.append(ssp_i)
    
FWHM_dif = np.sqrt(FWHM_gal**2 - FWHM_tem**2)
sigma = FWHM_dif/2.355/h2['CDELT1']  # Sigma difference in pixels
    
ssps = Table(
    map(
        list, zip(*ssp_rows)),
    names=['template name', 'spectral range', 'IMF type', 'IMF slope', 'Z', 't'])
ssps.sort(['Z', 't'])

template_prefix = template_files[0].split('/')[0]
template_files = ssps['template name']

for j in range(len(template_files)):
    hdu = fits.open(template_prefix + '/' + template_files[j] + '.fits')
    ssp = hdu[0].data
    ssp = ndimage.gaussian_filter1d(ssp, sigma)
    sspNew, logLam2, velscale = util.log_rebin(lamRange2, ssp, velscale=velscale)
    templates[:, j] = sspNew

In [59]:
plt.close('all')

nT = len(np.unique(ssps['t']))
nZ = len(np.unique(ssps['Z']))

f, axarr = plt.subplots(nT, nZ, figsize = (16, 12), sharex = 'col', sharey = 'row')

for i, row in enumerate(ssps):
    template = row['template name']
    
    ax = axarr[i%nT, i//nT]
    plt.setp(ax.get_xticklabels(), fontsize=6)
    plt.setp(ax.get_yticklabels(), fontsize=6)

    if not ax.is_last_row():
        plt.setp(ax.get_xticklabels(), visible=False)
        ax.xaxis.set_tick_params(size=0)
    else:
        ax.set_xlabel(r'$Z$: {}'.format(row['Z']), size = 6)
    if not ax.is_first_col():
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.yaxis.set_tick_params(size=0)
    else:
        ax.set_ylabel(r'$\tau$: {}'.format(row['t']), size = 6)
        yticks = ax.yaxis.get_major_ticks()
    ax.plot(np.exp(logLam2), templates[:, i]/np.median(templates), linewidth = 0.25, c = 'b')
        
plt.tight_layout()
plt.subplots_adjust(hspace=0.25, wspace=0)
plt.savefig('templates.png', dpi = 300)

In [95]:
template = ssps[(ssps['t'] == 12.5893) * (ssps['Z'] == 0.22)]

hdu = fits.open(template_prefix + '/' + template_files[j] + '.fits')
ssp = hdu[0].data
ssp = ndimage.gaussian_filter1d(ssp, sigma)
template, logLam2, velscale = util.log_rebin(lamRange2, ssp, velscale=velscale)

plt.close('all')
z = 0.0167887
c = 299792.458
template /= np.median(template)
galaxy /= np.median(galaxy)
plt.plot(np.exp(logLam2 + z), template, label = 'template', linewidth = 0.5)
plt.plot(np.exp(logLam1), galaxy, label = 'galaxy', linewidth = 0.5)
plt.legend(loc = 'best')
plt.ylim([0., 2.])
plt.show()